# Data cleaning

Standardized column names, dropped duplicate rows, then reshaped both CSVs from wide (year columns) to long format (year, value rows). Combined into one file.

In [ ]:
import re
import pandas as pd
from pathlib import Path
out_dir = Path('..') / 'data' / 'processed'
out_dir.mkdir(parents=True, exist_ok=True)
raw_dir = Path('..') / 'data' / 'raw'
print('Output dir:', out_dir.resolve())

Processed output dir: C:\Users\DELL\Desktop\SriLanka-Agricultural-Insights\data\processed


In [ ]:
# normalize column names
def normalize_col(c):
    if not isinstance(c, str):
        return c
    s = c.strip()
    s = s.replace('(US $ mn)', 'us_dollars_mn')
    s = s.replace('$', 'dollar')
    s = re.sub(r'[^0-9a-zA-Z]+', '_', s)
    s = re.sub(r'__+', '_', s)
    s = s.strip('_')
    s = s.lower()
    return s

In [ ]:
# load CSVs
csvs = sorted(list(raw_dir.glob('*.csv')))
dfs = {}
for p in csvs:
    try:
        dfs[p.name] = pd.read_csv(p, low_memory=False)
    except Exception as e:
        print('Failed to read', p.name, e)
print('Loaded:', list(dfs.keys()))


Loaded: ['production_of_major_agricultural_crops.csv', 'volume_and_value_of_export_agriculture_crops.csv']


In [ ]:
# normalize columns, drop empty rows and duplicates
cleaned = {}
for name, df in dfs.items():
    new_cols = [normalize_col(c) for c in df.columns]
    df.columns = new_cols
    df = df.dropna(how='all')
    dup_before = df.duplicated().sum()
    df = df.drop_duplicates()
    dup_after = df.duplicated().sum()
    print(f'{name}: {df.shape}, removed {dup_before - dup_after} duplicates')
    cleaned[name] = df


production_of_major_agricultural_crops.csv: shape after clean (13, 8), duplicates removed 0
volume_and_value_of_export_agriculture_crops.csv: shape after clean (3, 15), duplicates removed 0


In [8]:
# check columns after cleaning
for name, df in cleaned.items():
    print(f'{name}:', list(df.columns))


production_of_major_agricultural_crops.csv columns: ['crop', '2006', '2007', '2008', '2009', '2010', '2011', '2012']

volume_and_value_of_export_agriculture_crops.csv columns: ['crop', 'volume_2006', 'value_2006us_dollars_mn', 'volume_2007', 'value_2007us_dollars_mn', 'volume_2008', 'value_2008us_dollars_mn', 'volume_2009', 'value2009_us_dollars_mn', 'volume_2010', 'value2010_us_dollars_mn', 'volume_2011', 'value_2011us_dollars_mn', 'volume_2012', 'value_2012us_dollars_mn']


In [9]:
# melt production file
prod_name = 'production_of_major_agricultural_crops.csv'
if prod_name in cleaned:
    dfp = cleaned[prod_name].copy()
    yrs = [c for c in dfp.columns if re.match(r'^\d{4}$', str(c))]
    if yrs:
        prod_long = dfp.melt(id_vars=['crop'] if 'crop' in dfp.columns else [], value_vars=yrs, var_name='year', value_name='value')
        prod_long['year'] = pd.to_numeric(prod_long['year'], errors='coerce').astype('Int64')
        prod_long['value'] = pd.to_numeric(prod_long['value'], errors='coerce')
        prod_long['metric'] = 'production'
        prod_long = prod_long[['crop','year','metric','value']]
        prod_long.to_csv(out_dir / 'production_long.csv', index=False)
        print('Wrote production_long.csv', prod_long.shape)
    else:
        print('No year-columns found in', prod_name)
else:
    print(prod_name, 'not found')

Wrote production_long.csv (91, 4)


In [10]:
# melt export file (volume and value separate)
exp_name = 'volume_and_value_of_export_agriculture_crops.csv'
if exp_name in cleaned:
    dfe = cleaned[exp_name].copy()
    vol_cols = [c for c in dfe.columns if c.startswith('volume_')]
    val_cols = [c for c in dfe.columns if c.startswith('value_')]
    parts = []
    if vol_cols:
        v = dfe.melt(id_vars=['crop'] if 'crop' in dfe.columns else [], value_vars=vol_cols, var_name='var', value_name='value')
        v['year'] = v['var'].str.extract(r'(\d{4})')[0].astype('Int64')
        v = v[['crop','year','value']]
        v['metric'] = 'volume'
        parts.append(v)
    if val_cols:
        w = dfe.melt(id_vars=['crop'] if 'crop' in dfe.columns else [], value_vars=val_cols, var_name='var', value_name='value')
        w['year'] = w['var'].str.extract(r'(\d{4})')[0].astype('Int64')
        w = w[['crop','year','value']]
        w['metric'] = 'value_usd_mn'
        parts.append(w)
    if parts:
        export_long = pd.concat(parts, ignore_index=True)
        export_long = export_long.dropna(subset=['value'], how='all')
        export_long.to_csv(out_dir / 'export_long.csv', index=False)
        print('Wrote export_long.csv', export_long.shape)
    else:
        print('No volume/value columns found in', exp_name)
else:
    print(exp_name, 'not found')

Wrote export_long.csv (36, 4)


In [12]:
# Combine long files into a single cleaned file
parts = []
p1 = out_dir / 'production_long.csv'
p2 = out_dir / 'export_long.csv'
if p1.exists():
    parts.append(pd.read_csv(p1))
if p2.exists():
    parts.append(pd.read_csv(p2))
if parts:
    combined = pd.concat(parts, ignore_index=True, sort=False)
    # standardize column order
    cols = ['crop','year','metric','value']
    combined = combined[[c for c in cols if c in combined.columns]]
    combined.to_csv(out_dir / 'cleaned_agriculture_data.csv', index=False)
    print('Wrote cleaned_agriculture_data.csv', combined.shape)
else:
    print('No parts to combine')


Wrote cleaned_agriculture_data.csv (127, 4)
